In [1]:
import pandas as pd
import os

In [2]:
!pip install sweetviz


In [3]:
OUTPUT_PATH: str = os.environ[
    "OUTPUT_PATH"
] if "OUTPUT_PATH" in os.environ else os.path.join("output")
BASE_DIR: str = os.path.join(OUTPUT_PATH, "diginetica")
DATASET_DIR: str = os.path.join(OUTPUT_PATH, "diginetica", "dataset")

BASE_DATASET_FILE : str = os.path.join(OUTPUT_PATH, "diginetica", "dataset-train-diginetica", 'train-item-views.csv')

df = pd.read_csv(BASE_DATASET_FILE, sep=';')
df.head()

,sessionId,userId,itemId,timeframe,eventdate
0,1,NaN,81766,526309,2016-05-09
1,1,NaN,31331,1031018,2016-05-09
2,1,NaN,32118,243569,2016-05-09
3,1,NaN,9654,75848,2016-05-09
4,1,NaN,32627,1112408,2016-05-09


In [4]:
df['eventdate'] = pd.to_datetime(df['eventdate'])

In [5]:
# importing sweetviz
import sweetviz as sv
#analyzing the dataset
advert_report = sv.analyze(df)
#display the report
advert_report.show_html('Advertising.html')


Report Advertising.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [6]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, explorative=True)
profile

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "/home/marlesson/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-5af8ce0c51a9>", line 3, in <module>
    profile
  File "/home/marlesson/.local/lib/python3.7/site-packages/IPython/core/displayhook.py", line 262, in __call__
    format_dict, md_dict = self.compute_format_data(result)
  File "/home/marlesson/.local/lib/python3.7/site-packages/IPython/core/displayhook.py", line 151, in compute_format_data
    return self.shell.display_formatter.format(result)
  File "/home/marlesson/.local/lib/python3.7/site-packages/IPython/core/formatters.py", line 180, in format
    data = formatter(obj)
  File "</home/marlesson/.local/lib/python3.7/site-packages/decorator.py:decorator-gen-9>", line 2, in __call__
  File "/home/marlesson/.local/lib/python3.7/site-packages/IPython/core/formatters.py", line 224, in catch_format_error
    r 

KeyboardInterrupt: 

## Spark

In [ ]:
list_sum = sum

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import collect_set, collect_list, lit, sum, udf, concat_ws, col, count, abs, date_format, \
    from_utc_timestamp, expr, min, max
from pyspark.sql.functions import col, udf, size
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import explode, posexplode
from datetime import datetime, timedelta

pad_history = F.udf(
    lambda arr, size: list(reversed(([0] * (size - len(arr[:-1][:size])) + arr[:-1][:size]))), 
    ArrayType(IntegerType())
)

def sample_items(item, item_list, size_available_list):
    return random.sample(item_list, size_available_list - 1) + [item]

def udf_sample_items(item_list, size_available_list):
    return udf(lambda item: sample_items(item, item_list, size_available_list), ArrayType(IntegerType()))


def concat(type):
    def concat_(*args):
        return list(set(chain.from_iterable((arg if arg else [] for arg in args))))
    return udf(concat_, ArrayType(type))


OUTPUT_PATH: str = os.environ[
    "OUTPUT_PATH"
] if "OUTPUT_PATH" in os.environ else os.path.join("output")
BASE_DIR: str = os.path.join(OUTPUT_PATH, "diginetica")
DATASET_DIR: str = os.path.join(OUTPUT_PATH, "diginetica", "dataset")

BASE_DATASET_FILE : str = os.path.join(OUTPUT_PATH, "diginetica", "dataset-train-diginetica", 'train-item-views.csv')


In [ ]:
df = spark.read.option("delimiter", ";").csv(BASE_DATASET_FILE, header=True, inferSchema=True)
df = df.withColumnRenamed("sessionId", "SessionID")\
    .withColumnRenamed("eventdate", "Timestamp")\
    .withColumnRenamed("itemId", "ItemID")\
    .withColumn("Timestamp", (col("Timestamp").cast("long") + col("timeframe").cast("long")/1000).cast("timestamp"))\
    .orderBy(col('Timestamp'), col('SessionID'), col('timeframe')).select("SessionID", "ItemID", "Timestamp", "timeframe")


In [ ]:
df.show(10, False), df.count()

In [ ]:
# Drop duplicate item in that same session
df = df.dropDuplicates(['SessionID', 'ItemID'])
df.count()

In [ ]:
# Filtra
sample_days = 8
minimum_interactions = 5

# filter date
max_timestamp  = df.select(max(col('Timestamp'))).collect()[0]['max(Timestamp)']
init_timestamp = max_timestamp - timedelta(days = sample_days)
df             = df.filter(col('Timestamp') >= init_timestamp).cache()


# Filter minin interactions
df_item    = df.groupBy("ItemID").count()
df_item    = df_item.filter(col('count') >= minimum_interactions)


# Filter session size
df_session    = df.groupBy("SessionID").count()
df_session    = df_session.filter(col('count') >= 2)

df = df \
    .join(df_item, "ItemID", how="inner") \
    .join(df_session, "SessionID", how="inner")

df.orderBy(col('SessionID'), col('Timestamp')).show(10, False), df.count()

# return df

In [ ]:
df_item    = df.groupBy("ItemID").count()
df_item.count()

In [ ]:
# Add history
history_window = 10
##

w = Window.partitionBy('SessionID').orderBy('Timestamp')#.rangeBetween(Window.currentRow, 5)

df = df.withColumn(
    'ItemIDHistory', F.collect_list('ItemID').over(w)
).where(size(col("ItemIDHistory")) >= 2)#\

df = df.withColumn('ItemIDHistory', pad_history(df.ItemIDHistory, lit(history_window)))
df.orderBy(col('SessionID'), col('Timestamp')).show(10, False), df.count()


In [ ]:
df = df.toPandas()
df.head()

In [ ]:
df = df.sort_values('Timestamp')

In [ ]:
days_test = 1
timestamp_property = 'Timestamp'

In [ ]:
import pandas as pd
cutoff_date = df[timestamp_property].iloc[-1] - pd.Timedelta(days=days_test)
cutoff_date, df[timestamp_property].iloc[-1]

In [ ]:
df[df[timestamp_property] >= cutoff_date].head()

In [ ]:
df_test = pd.read_csv('/media/workspace/triplet_session/output/diginetica/dataset/test_0.20_test=time_42_SessionInteractionDataFrame_____SessionID_6cdc23187f.csv')
df_test.shape

In [ ]:
df_test.head()

## Spark session

In [ ]:
#parans
min_itens_per_session  = 2
max_itens_per_session  = 15
min_itens_interactions = 3 # Tupla interactions
max_relative_pos       = 2

In [ ]:
df = spark.read.option("delimiter", ";").csv(BASE_DATASET_FILE, header=True, inferSchema=True)
df = df.withColumnRenamed("sessionId", "SessionID")\
    .withColumnRenamed("eventdate", "Timestamp")\
    .withColumnRenamed("itemId", "ItemID")\
    .withColumn("Timestamp", (col("Timestamp").cast("long") + col("timeframe").cast("long")/1000).cast("timestamp"))\
    .orderBy(col('Timestamp'), col('SessionID'), col('timeframe')).select("SessionID", "ItemID", "Timestamp", "timeframe")
df.show(10)

In [ ]:
# Drop duplicate item in that same session
df       = df.dropDuplicates(['SessionID', 'ItemID'])

In [ ]:
# Filtra
sample_days = 8
minimum_interactions = 5

# filter date
max_timestamp  = df.select(max(col('Timestamp'))).collect()[0]['max(Timestamp)']
init_timestamp = max_timestamp - timedelta(days = sample_days)
df             = df.filter(col('Timestamp') >= init_timestamp).cache()

In [ ]:
df       = df.groupby("SessionID").agg(
            max("Timestamp").alias("Timestamp"),
            collect_list("ItemID").alias("ItemIDs"),
            count("ItemID").alias("total"))

df.show(10), df.count()

In [ ]:
# Filter Interactions
df = df.filter(df.total >= min_itens_per_session)

In [ ]:
df_pos = df.select(col('SessionID').alias('_SessionID'),
                            posexplode(df.ItemIDs))

In [ ]:
df_pos.show(10), df_pos.count()

In [ ]:
# Explode A
df = df.withColumn("ItemID_A", explode(df.ItemIDs))
df = df.join(df_pos,
            (df.SessionID == df_pos._SessionID) & (df.ItemID_A == df_pos.col))\
        .select('SessionID', 'Timestamp', 'ItemID_A', 'pos', 'ItemIDs')\
        .withColumnRenamed('pos', 'pos_A')

df.show(10), df.count()

In [ ]:
# Explode B
df = df.withColumn("ItemID_B", explode(df.ItemIDs))
df = df.join(df_pos,
            (df.SessionID == df_pos._SessionID) & (df.ItemID_B == df_pos.col))\
        .withColumnRenamed('pos', 'pos_B')
df.show(10), df.count()

In [ ]:
df = df.withColumn("relative_pos", abs(df.pos_A - df.pos_B))
df.show(10), df.count()

In [ ]:
df = df.select('SessionID', 'Timestamp', 'ItemID_A', 'pos_A', 
                        'ItemID_B', 'pos_B', 'relative_pos')\
                .distinct()\
                .filter(df.ItemID_A != df.ItemID_B)
df.show(10), df.count()

In [ ]:
# get_df_tuple_probs

df_tuple_count  = df.groupby("ItemID_A","ItemID_B").count()
df_count        = df.groupby("ItemID_A").count()\
                    .withColumnRenamed("count", "total")\
                    .withColumnRenamed("ItemID_A", "_ItemID_A")

df_join         = df_tuple_count.join(df_count, df_tuple_count.ItemID_A == df_count._ItemID_A)
df_join         = df_join.withColumn("prob", col("count")/col("total"))

df_join  = df_join.select("ItemID_A", 'ItemID_B', 'count', 'total', 'prob')\
            .withColumnRenamed("ItemID_A", "_ItemID_A")\
            .withColumnRenamed("ItemID_B", "_ItemID_B")\
            .withColumnRenamed("count", "total_ocr_dupla")\
            .withColumnRenamed("total", "total_ocr")

df_join.show(10), df_join.count()

In [ ]:
df_probs = df_join
df = df.join(df_probs, (df.ItemID_A == df_probs._ItemID_A) & (df.ItemID_B == df_probs._ItemID_B))
df.show(10), df.count()

In [ ]:
import numpy as np
#add_positive_interactions
pos_max_deep = 1

def serach_positive(uid, df, max_deep = 1, deep = 0, list_pos = []):
    if uid not in list_pos:
        list_pos.append(uid)
        #print(list_pos)
       
        if deep >= max_deep:
            return df.loc[uid].sub_a_b
        else:
            l = [serach_positive(i, df, max_deep, deep+1, list_pos) for i in df.loc[uid].sub_a_b]
            l.append(df.loc[uid].sub_a_b)
            return list_sum(l, [])
    else:
        return []
    
    
# Filter more then 1 ocurrence for positive interactions
df_positive = df.filter(col("total_ocr_dupla") > 1)

df_positive = df_positive\
    .groupby("ItemID_A")\
    .agg(F.collect_set("ItemID_B").alias("sub_a_b"))

# df_b = df\
#     .groupby("ItemID_B")\
#     .agg(F.collect_set("ItemID_A").alias("sub_b"))

# df = df.join(df_a, "ItemID_A").join(df_b, "ItemID_B").cache()

# concat_int_arrays = concat(IntegerType())
# df = df.withColumn("sub_a_b", concat_int_arrays("sub_a", "sub_b"))#.show(truncate=False)
# return df
df_positive = df_positive.withColumnRenamed("ItemID_A", "ItemID")
#df = df.withColumn("ItemID_COPY",df.ItemID)

df_positive = df_positive.toPandas().set_index('ItemID')
print(df_positive)

sub_pos = []
for i, row in df_positive.iterrows():
    l = serach_positive(row.name, df_positive, max_deep = pos_max_deep, deep=0, list_pos=[])
    sub_pos.append(list(np.unique(l)))

df_positive['sub_a_b_all'] = sub_pos
df_positive

In [ ]:
df_positive.loc[381966].sub_a_b

In [ ]:
df = df.filter(col("total_ocr_dupla") >= min_itens_interactions)\
       .filter(col("relative_pos") <= max_relative_pos)\
       .filter(col("pos_A") <= max_itens_per_session)

df = df.select("SessionID", 'Timestamp', 'ItemID_A', 'ItemID_B', 'relative_pos', 
                        'total_ocr', 'total_ocr_dupla')

df.show(10), df.count()

In [ ]:
df = df.dropDuplicates(['ItemID_A', 'ItemID_B', 'relative_pos'])
df.count()